In [132]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from sklearn.model_selection import train_test_split
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


## 1) Getting The Data

### First we'll analyze the dataset given to us 

In [133]:
train_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/train.csv'
test_file_path = '/kaggle/input/house-prices-advanced-regression-techniques/test.csv'
df_train = pd.read_csv(train_file_path)
df_test = pd.read_csv(test_file_path)

In [134]:
df_train

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [135]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,...,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,6,2006,WD,Normal
1455,2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml
1456,2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml
1457,2918,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal


## 2) Training and Validation DataSet

In [136]:
X_train = df_train.drop('SalePrice', axis=1)
Y_train = df_train['SalePrice']

X_test = df_test[df_train.drop('SalePrice', axis=1).columns]

- we have splitted the data set to make sure our model first works properly and gives accurate predictions on the validation set and then we can move on to the testing dataset

In [137]:
X_train, X_validation, Y_train, Y_validation = train_test_split(X_train, Y_train, test_size=0.2, random_state=42)

## 3) One Hot Encoding

#### as you can see above there are not all values which we Numerical, so we'll use ONE HOT ENCODING to turn the non numerical values into numerical values

In [138]:
X_train_encoded = pd.get_dummies(X_train)
X_validation_encoded = pd.get_dummies(X_validation)

X_test_encoded = pd.get_dummies(X_test)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
X_test_encoded = X_test_encoded.dropna(axis=0)
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
# Drop rows with NaN values from both training and validation sets
X_train_encoded = X_train_encoded.dropna()
Y_train = Y_train.loc[X_train_encoded.index]

X_validation_encoded = X_validation_encoded.dropna()
Y_validation = Y_validation.loc[X_validation_encoded.index]

X_train_encoded, X_validation_encoded = X_train_encoded.align(X_validation_encoded, axis=1, fill_value=0)

- Here we splitted the data into two sets so we can check if our model predicts properly and gives right prediction with less errors and then we can put it on the main training set so we can get our predictions

## 4) Linear Regression Model

In [139]:
Linear_Reg = LinearRegression()
Linear_Reg.fit(X_train_encoded,Y_train)

LinearRegression()

- Now we'll do some predictions

In [140]:
predictions = Linear_Reg.predict(X_validation_encoded)

In [141]:
predictions_df = pd.DataFrame({'Id': Y_validation.index, 'SalePrice': predictions})

print("ORIGINAL VALUE: ")
print(Y_validation.head())
print("PREDICTED VALUE: ")

for index, row in predictions_df.iterrows():
    print(f"Id: {row['Id']}, Predicted SalePrice: {row['SalePrice']}")
    if index==4:
        break


ORIGINAL VALUE: 
892     154500
1105    325000
413     115000
522     159000
1036    315500
Name: SalePrice, dtype: int64
PREDICTED VALUE: 
Id: 892.0, Predicted SalePrice: 164097.87719726562
Id: 1105.0, Predicted SalePrice: 340129.2199707031
Id: 413.0, Predicted SalePrice: 90978.56030273438
Id: 522.0, Predicted SalePrice: 179057.59008789062
Id: 1036.0, Predicted SalePrice: 318790.7180175781


- as you can see above the predction is near to the original price of the houses and hence we can go ahead with the test.csv

## 5) Mean Absolute Error

In [142]:
mae = mean_absolute_error(Y_validation, predictions)
error_percentage = (mae / Y_validation.mean()) * 100
print(mae)
print(error_percentage)

15447893178.965712
8513823.743281437


- we are just calculating the mean absolute error for just the formality 

## 6) Predictions on Test DataSet

In [143]:
X_test_encoded = X_test_encoded.reindex(columns=X_train_encoded.columns, fill_value=0)
predictions_test = Linear_Reg.predict(X_test_encoded)

In [144]:
# Assuming predictions_test contains your predicted values and df_test contains your test data
# Ensure that the lengths match by taking a subset of df_test['Id']
predictions_df_test = pd.DataFrame({'Id': df_test['Id'].head(len(predictions_test)), 'SalePrice': predictions_test})

print("PREDICTED VALUES FOR TEST DATA: ")
for index, row in predictions_df_test.iterrows():
    # Skip the second iteration
    if index == 21:
        break
    if index == 1:  
        continue
    print(f"Id: {row['Id']}, Predicted SalePrice: {row['SalePrice']}")


PREDICTED VALUES FOR TEST DATA: 
Id: 1461.0, Predicted SalePrice: 128535.8359375
Id: 1463.0, Predicted SalePrice: 186502.54174804688
Id: 1464.0, Predicted SalePrice: 187550.96728515625
Id: 1465.0, Predicted SalePrice: 216872.94995117188
Id: 1466.0, Predicted SalePrice: 168671.78344726562
Id: 1467.0, Predicted SalePrice: 153446.40673828125
Id: 1468.0, Predicted SalePrice: 201518.89672851562
Id: 1469.0, Predicted SalePrice: 115962.14892578125
Id: 1470.0, Predicted SalePrice: 159565.505859375
Id: 1471.0, Predicted SalePrice: 105674.203125
Id: 1472.0, Predicted SalePrice: 101925.60131835938
Id: 1473.0, Predicted SalePrice: 142459.34057617188
Id: 1474.0, Predicted SalePrice: 104998.9892578125
Id: 1475.0, Predicted SalePrice: 370002.2880859375
Id: 1476.0, Predicted SalePrice: 267326.4638671875
Id: 1477.0, Predicted SalePrice: 310467.03857421875
Id: 1478.0, Predicted SalePrice: 303030.826171875
Id: 1479.0, Predicted SalePrice: 493145.4885253906
Id: 1480.0, Predicted SalePrice: 312929.12890625